# Mini project: Simulating Pac-Man

In the last practical session, we saw how run multiple behaviors in parallel on multiple robots. In this section we will see more advanced methods for combining behaviors by modulating behavior activations according to internal states of the robot and by allowing them to sense the attributes of others. In order to start with a clean basis, let's first provide the definition of several behaviors. These four behaviors are simple implementations of the [Braitenberg vehicles](https://cdn.rawgit.com/clement-moulin-frier/rti_course/master/class_1/intro.sozi.html#frame1550) we have seen in class, where the sources (i.e. what is sensed by the proximeters) are other epucks:

In [ ]:
from simulator_interface import open_session, close_session
simulator, epuck = open_session()

# First reduce the maximum speed
epuck.max_speed = 1.5

def line_following(epuck):
    floor_left, floor_middle, floor_right = epuck.floor_sensor()
    if floor_middle == 1.0 and floor_right == 1.0:
        # There's a left turn
        return  [1.0, 0]
    if floor_middle == 1.0: return [1, 1]
    if floor_left == 0.0 and floor_right == 0.0: return [1, 0]
    return [floor_right, floor_left] 

epuck.detach_all_behaviors()
epuck.attach_behavior(line_following, freq=10)
epuck.start_all_behaviors()
    